In [5]:
from transformers import AutoTokenizer, Trainer ,BertForSequenceClassification
from datasets import load_dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# define output dir
FULL_FINETUNING_OUTPUT_DIR = "/content/drive/My Drive/Colab Notebooks/output/distilbert-qa-full-finetuned"
LORA_FINETUNING_OUTPUT_DIR = "/content/drive/My Drive/Colab Notebooks/output/distilbert-qa-lora-finetuned"

In [8]:
# define tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")

#load dataset
ds = load_dataset("lansinuote/ChnSentiCorp")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0

dataset_infos.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

...

data/validation-00000-of-00001-405befbaa(…):   0%|          | 0.00/276k [00:00<?, ?B/s]

data/test-00000-of-00001-5372924f059fe76(…):   0%|          | 0.00/275k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [9]:
#data cleaning
import re

def clean_text(text):
  text = re.sub(r'[^\w\s]', '',text)
  text = text.strip()
  return text

ds = ds.map(lambda x: {'text': clean_text(x['text'])})

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [10]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/9600 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

In [11]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1200
    })
})


## prepare dataset


In [12]:
train_dataset = tokenized_datasets['train']
val_dataset = tokenized_datasets['validation']
test_dataset = tokenized_datasets['test']

## define training parameter



In [13]:
from transformers import TrainingArguments

training_args_full = TrainingArguments(
    output_dir=FULL_FINETUNING_OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch"
)

training_args_lora = TrainingArguments(
    output_dir=LORA_FINETUNING_OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch"
)

## LoRA Finetuning


In [14]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
)

# Get the LoRA model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


## Train with LoRA

In [25]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

In [16]:
trainer_lora = Trainer(
    model=lora_model,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_lora.train()

trainer_lora.save_model(LORA_FINETUNING_OUTPUT_DIR)
print(f"Model saved to {LORA_FINETUNING_OUTPUT_DIR}")

/tmp/ipython-input-4094437312.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_lora = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: wuuu111 (wuuu111-nus) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.702100,0.688436,0.585000,0.584009
2,0.661500,0.649642,0.638333,0.625380
3,0.604700,0.587101,0.703333,0.703284
4,0.578800,0.581082,0.705833,0.705791
5,0.593300,0.579470,0.700833,0.700681


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Model saved to /content/drive/My Drive/Colab Notebooks/output/distilbert-qa-lora-finetuned


## Full Finetuning


In [17]:
from transformers import Trainer

trainer_full = Trainer(
    model=model,
    args=training_args_full,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer_full.train()

trainer_full.save_model(FULL_FINETUNING_OUTPUT_DIR)
print(f"Model saved to {FULL_FINETUNING_OUTPUT_DIR}")

/tmp/ipython-input-2381540415.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.635400,0.578336,0.703333,0.702524
2,0.537900,0.599135,0.700833,0.695140
3,0.544000,0.571943,0.707500,0.707490


Model saved to /content/drive/My Drive/Colab Notebooks/output/distilbert-qa-full-finetuned


## evaluation with test dataset
Evaluate the full finetuning and LoRA models on the test set using accuracy and F1 score.

In [18]:
# Evaluate full finetuning model on the test set
print("Evaluating full finetuning model on the test set...")
eval_results_full = trainer_full.evaluate(test_dataset)
print(f"Full finetuning model evaluation results:")
print(f"  Accuracy: {eval_results_full.get('eval_accuracy', 'N/A')}")
print(f"  F1 Score: {eval_results_full.get('eval_f1', 'N/A')}")



Evaluating full finetuning model on the test set...


Full finetuning model evaluation results:
  Accuracy: 0.7141666666666666
  F1 Score: 0.7141744088947476


In [22]:
from peft import PeftModel
from transformers import Trainer
from transformers import BertForSequenceClassification

# Load the base model
lora_base_model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")
lora_model_merged = PeftModel.from_pretrained(lora_base_model, LORA_FINETUNING_OUTPUT_DIR)

# Get the base model with merged LoRA weights
merged_base_model = lora_model_merged.merge_and_unload()


# Create a new Trainer for the merged model
trainer_lora_merged = Trainer(
    model=merged_base_model,
    args=training_args_lora,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Evaluate the merged model using the new Trainer
eval_results_lora = trainer_lora_merged.evaluate()
print(f"LoRA model evaluation results:")
print(f"  Accuracy: {eval_results_lora.get('eval_accuracy', 'N/A')}")
print(f"  F1 Score: {eval_results_lora.get('eval_f1', 'N/A')}")

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.



Evaluating LoRA model on the test set...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert.embeddings.LayerNorm.bias', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.token_type_embeddings.weight', 'bert.embeddings.word_embeddings.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.outp

LoRA model evaluation results:
  Accuracy: 0.5208333333333334
  F1 Score: 0.3832644730320391
